In [1]:
! pip install prophet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
   import pkg_resources
   installed_packages = pkg_resources.working_set
   installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
   print(installed_packages_list)
   

['altair==5.5.0', 'anyio==4.6.2.post1', 'argon2-cffi-bindings==21.2.0', 'argon2-cffi==23.1.0', 'arrow==1.3.0', 'asttokens==2.4.1', 'async-lru==2.0.4', 'attrs==24.2.0', 'autocommand==2.2.2', 'babel==2.16.0', 'backports.tarfile==1.2.0', 'beautifulsoup4==4.12.3', 'bleach==6.1.0', 'blinker==1.9.0', 'branca==0.8.1', 'cachetools==5.5.0', 'category-encoders==2.8.1', 'certifi==2024.8.30', 'cffi==1.17.1', 'chardet==5.2.0', 'charset-normalizer==3.4.0', 'click==8.1.7', 'cmdstanpy==1.2.5', 'colorama==0.4.6', 'comm==0.2.2', 'contourpy==1.3.1', 'cycler==0.12.1', 'dash-core-components==2.0.0', 'dash-html-components==2.0.0', 'dash-table==5.0.0', 'dash==2.18.2', 'debugpy==1.8.7', 'decorator==5.1.1', 'defusedxml==0.7.1', 'et-xmlfile==2.0.0', 'executing==2.1.0', 'fastjsonschema==2.20.0', 'flask-cors==6.0.1', 'flask==3.0.3', 'folium==0.19.4', 'fonttools==4.55.2', 'fqdn==1.5.1', 'geographiclib==2.0', 'geopy==2.4.1', 'gitdb==4.0.11', 'gitpython==3.1.43', 'h11==0.14.0', 'holidays==0.70', 'httpcore==1.0.6', '

C:\Users\ICONS\AppData\Local\Temp\ipykernel_3436\2772129146.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [1]:
%%writefile local.py
import streamlit as st
import pandas as pd
import plotly.express as px
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

# 1. App Title
st.title("🏨 Hotel Performance Analysis – Local and International Markets")

# 2. Upload CSV
uploaded_file = st.sidebar.file_uploader("📤 Upload Cleaned_Hotel_Booking.csv", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv("https://raw.githubusercontent.com/USERNAME/REPO/main/Cleaned_Hotel_Booking.csv")


    # 3. Data Cleaning and Preprocessing
    df['Arrival date'] = pd.to_datetime(df['Arrival date'], errors='coerce')
    df['Year'] = df['Arrival date'].dt.year
    df['Month'] = df['Arrival date'].dt.month
    df['YearMonth'] = df['Arrival date'].dt.to_period('M').astype(str)
    df['Market_Type'] = df['Country'].apply(lambda x: 'Local' if str(x).strip().lower() == 'egypt' else 'International')

    st.subheader("🧾 Data Sample")
    st.write(df.head())

    # 4. Bookings 2018 (Bar Chart)
    df_2018 = df[df['Year'] == 2018]
    pivot_2018 = df_2018.groupby(['Month', 'Market_Type'])['No Of Rooms'].sum().reset_index()
    fig1 = px.bar(pivot_2018, x='Month', y='No Of Rooms', color='Market_Type', barmode='group',
                  title='📊 Local and International Bookings in 2018')
    st.plotly_chart(fig1)

    # 5. Occupancy Rate (Line Chart)
    occupancy = df.groupby(['YearMonth', 'Market_Type'])['Occupancy Rate %'].mean().reset_index()
    fig2 = px.line(occupancy, x='YearMonth', y='Occupancy Rate %', color='Market_Type',
                   title='📈 Monthly Occupancy Rate by Market')
    st.plotly_chart(fig2)

    # 6. ADR & RevPAR (Table)
    st.subheader("📌 Average ADR and RevPAR by Market")
    adr_revpar = df.groupby('Market_Type')[['ADR', 'RevPAR']].mean().reset_index()
    st.dataframe(adr_revpar)

    # 7. Monthly Revenue (Area Chart)
    market_monthly = df.groupby(['YearMonth', 'Market_Type'])['total rate net'].sum().reset_index()
    fig3 = px.area(market_monthly, x='YearMonth', y='total rate net', color='Market_Type',
                   title='📊 Monthly Revenue – Local vs International',
                   template='plotly_white', groupnorm='fraction')
    st.plotly_chart(fig3)

    # 8. Revenue Share (Pie Chart)
    market_rev = df.groupby('Market_Type')['total rate net'].sum().reset_index()
    fig4 = px.pie(market_rev, names='Market_Type', values='total rate net', hole=0.5,
                  title='🎯 Market Share by Revenue')
    st.plotly_chart(fig4)

    # 9. Prophet Forecast – Local Bookings
    st.subheader("📅 Forecasting Local Market Bookings")
    monthly_local = df[df['Market_Type'] == 'Local'].groupby('YearMonth')['No Of Rooms'].sum().reset_index()
    monthly_local['ds'] = pd.to_datetime(monthly_local['YearMonth'])
    monthly_local['y'] = monthly_local['No Of Rooms']
    monthly_local = monthly_local[['ds', 'y']]

    model = Prophet()
    model.fit(monthly_local)
    future = model.make_future_dataframe(periods=6, freq='ME')  # Use 'ME' to avoid FutureWarning
    forecast = model.predict(future)

    fig5 = px.line(forecast, x='ds', y='yhat', labels={'ds': 'Date', 'yhat': 'Expected Room Bookings'},
                   title='🔮 Forecast: Local Market Bookings (Next 6 Months)')
    st.plotly_chart(fig5)

    # 10. ML Model – Predict ADR
    st.subheader("🧠 Predict ADR (Average Daily Rate) using Machine Learning")
    ml_df = df.dropna(subset=['ADR'])

    # Features and Target
    X = ml_df[['No Of Rooms', 'Adult', 'Child', 'T.Pax', 'Room Type', 'Market_Type', 'arrival_day', 'arrival_month', 'booking_day_diff']]
    y = ml_df['ADR']

    cat_features = ['Room Type', 'Market_Type']
    num_features = ['No Of Rooms', 'Adult', 'Child', 'T.Pax', 'arrival_day', 'arrival_month', 'booking_day_diff']

    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', 'passthrough', num_features)
    ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    st.success(f"✅ Model Trained – MAE: {round(mae, 2)}")

    # 11. User Input for Prediction
    st.markdown("### 🔍 Try a Custom Prediction")
    user_input = {
        'No Of Rooms': st.number_input('No Of Rooms', min_value=1, max_value=10, value=2),
        'Adult': st.number_input('Adults', min_value=1, max_value=10, value=2),
        'Child': st.number_input('Children', min_value=0, max_value=10, value=0),
        'T.Pax': st.number_input('Total Pax', min_value=1, max_value=20, value=2),
        'Room Type': st.selectbox('Room Type', options=df['Room Type'].dropna().unique()),
        'Market_Type': st.selectbox('Market', options=['Local', 'International']),
        'arrival_day': st.number_input('Arrival Day', min_value=1, max_value=31, value=15),
        'arrival_month': st.number_input('Arrival Month', min_value=1, max_value=12, value=6),
        'booking_day_diff': st.number_input('Booking Lead Time (days)', min_value=0, max_value=365, value=30)
    }

    user_df = pd.DataFrame([user_input])
    prediction = pipeline.predict(user_df)[0]
    st.success(f"🔮 Predicted ADR: {round(prediction, 2)} EGP")

else:
    st.info("📥 Please upload your cleaned hotel booking CSV file to begin.")


Overwriting local.py


In [ ]:
! streamlit run local.py